<a href="https://colab.research.google.com/github/pdrobny/Potential_Talents/blob/main/HF_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace with Gemma






## Setup

In [ ]:
!pip install transformers sentence-transformers
!pip install transformers torch
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import warnings
import logging
import random
import requests
import sys
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer, set_seed
from huggingface_hub import notebook_login
warnings.filterwarnings('ignore', category=UserWarning)

print(torch.__version__)
#tf.__version__

2.6.0+cu124


# Data prep

In [ ]:
df = pd.read_csv('talents.csv')
df

,id,title,sentence_bert_cossim
0,1,innovative and driven professional seeking a r...,1.000000
1,431,aspiring data science professional focused on ...,0.769162
2,544,data analyst data scientist business analyst d...,0.768222
3,833,data analyst turning complex data into actiona...,0.745245
4,199,ms in information systems northeastern univers...,0.727268
...,...,...,...
1260,648,research specialist university of rochester di...,0.079923
1261,730,medical biller at brick pediatric group,0.072848
1262,990,ingeniero elctrico,0.067254
1263,296,company owner at armstrong cleans carpets,0.056890


# Hugging Face

## Prompting

In [ ]:
job_titles = df["title"].tolist()


In [ ]:
job_titles_short = df["title"].head(10).tolist()
job_titles_short

['innovative and driven professional seeking a role in data analyticsdata science in the information technology industry.',
 'aspiring data science professional focused on data analysis machine learning and data visualization actively seeking opportunities',
 'data analyst data scientist business analyst driving data-driven insights strategic solutions',
 'data analyst turning complex data into actionable insights passionate about solving business challenges with data-driven solutions',
 'ms in information systems northeastern university data scientist business intelligence data engineering data analyst transforming data into insights',
 'aspiring data scientist passion for data-driven decision making master of science in business analytics graduate  university of new hampshire',
 'data scientist and analyst driving business insights with advanced data techniques research expertise',
 'master of science in analytics at georgia institute of technology aspiring data scientist',
 'passion

In [ ]:
job_ids_short = df["id"].head(10).tolist()
job_ids_short

[1, 431, 544, 833, 199, 28, 1282, 426, 963, 487]

In [ ]:
target_title = "data analyst"

In [ ]:
notebook_login()

In [ ]:
model_id = "google/gemma-3-1b-it"
generator = pipeline('text-generation', model=model_id)
set_seed(42)
prompt = f"""
Return a list of the top 5 job candidates with full job title and job id from a job titles list ranked by their similirality to the search term in desecnding order.
**Search term**
{target_title}

**job titles**
{job_titles_short}

**job ids**
{job_ids_short}


Show answer in following format:
1 - [job id]: [job title]
...
Answer: Top 5 are:
"""
output = generator(prompt, max_new_tokens=200, num_return_sequences=1)
print(output[0],['generated_text'])

Device set to use cpu


{'generated_text': '\nReturn a list of the top 5 job candidates with full job title and job id from a job titles list ranked by their similirality to the search term in desecnding order.\n**Search term**\ndata analyst\n\n**job titles**\n[\'innovative and driven professional seeking a role in data analyticsdata science in the information technology industry.\', \'aspiring data science professional focused on data analysis machine learning and data visualization actively seeking opportunities\', \'data analyst data scientist business analyst driving data-driven insights strategic solutions\', \'data analyst turning complex data into actionable insights passionate about solving business challenges with data-driven solutions\', \'ms in information systems northeastern university data scientist business intelligence data engineering data analyst transforming data into insights\', \'aspiring data scientist passion for data-driven decision making master of science in business analytics gradua

In [ ]:
generated_text = output[0]['generated_text']
answer_start_index = generated_text.find("Answer:")
# Extract the answer and print it
if answer_start_index != -1:
    # Add the length of the search string to get the true start of the answer
    answer_start_index += len("Answer:")
    print(generated_text[answer_start_index:].strip())
else:
    # If the marker was not found, print the whole output or a message
    print("Could not find the start of the answer in the generated text.")
    print(generated_text)

Top 5 are:
1 - 431: innovative and driven professional seeking a role in data analytics
2 - 544: data scientist and analyst driving business insights with advanced data techniques research expertise
3 - 833: data scientist and analyst driving business insights with advanced data techniques research expertise
4 - 199: data analyst turning complex data into actionable insights passionate about solving business challenges with data-driven solutions
5 - 28: aspiring data scientist seeking opportunities in the data field data analyst with experience at sritech software expertise in machine learning data evaluation passionate about transforming data into insights
**Explanation:**

The provided job titles are ranked by their similarity to the search term "data analyst." The job IDs are the corresponding job ids. The code should return a list of the top 5 job candidates with full job title and job id from a job titles list ranked by their similiarity to the search term in descending order.

``